In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import nltk

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install accelerate

In [ ]:
!pip install rouge_score

In [ ]:
# import transformers
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, load_from_disk, Dataset
from sklearn.model_selection import train_test_split

In [ ]:
import accelerate
import torch

In [ ]:
import transformers
import accelerate
import torch

print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)
print("PyTorch version:", torch.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv("/content/drive/My Drive/preprocessed_recipe_dataset.csv")

In [17]:
data.shape
# print(data['answer'][0])

(1000, 2)

In [ ]:
df = pd.DataFrame(data)
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# max_input = 512
# max_target = 128
# batch_size = 3
max_length = 512
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large')

In [ ]:
def preprocess_function(examples):
    inputs = [q for q in examples['question']]
    model_inputs = tokenizer(inputs,padding='max_length', max_length=max_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], padding='max_length', max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

In [ ]:
# from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=2,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    gradient_accumulation_steps=8,
    learning_rate=5e-6,
)

In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")

def compute_rouge(predictions, references):
    # Decode predictions and references if they are token IDs
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True) for pred in predictions.predictions]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True, clean_up_tokenization_spaces=True) for label in references.label_ids]

    # Sometimes you have to remove padding or other special tokens manually
    decoded_labels = ["\n".join([ll for ll in label.splitlines() if len(ll) > 0]) for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return {"rouge": result}


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
trainer.train()

In [14]:
trainer.evaluate()

KeyboardInterrupt: 

In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')


In [ ]:
# from transformers import BartForConditionalGeneration, BartTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained('./saved_model')
tokenizer = AutoTokenizer.from_pretrained('./saved_model')

# Encode input context
input_text = "What dishes can we make with beef and tomato?"
inputs = tokenizer(input_text, return_tensors="pt")

# # Generate a response
# outputs = model.generate(inputs['input_ids'], num_beams=5, early_stopping=True)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Debug: Check input tokens
# print("Tokenized inputs:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))

outputs = model.generate(**inputs, max_length=max_length)
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Debug: Check generated text
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
